In [1]:
# Import necessary libraries
import os
from datetime import datetime
import rasterio
from tqdm import tqdm
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape
import json
from tensorflow import keras
from utils import *
from leaf_prediction import *
from mrcnn import model as modellib
from mrcnn.config import Config
from pathlib import Path



# Define matrices F1 and F4
F1 = np.array([[1, 1, 1], [1, 1, 1], [1, 1, 1]])
F4 = np.array([
    [1, 1, 1, 1, -4, 1, 1, 1, 1],
    [1, 1, -3.6, -3.2, -3, -3.2, -3.6, 1, 1],
    [1, -3.6, -2.8, -2.2, -2, -2.2, -2.8, -3.6, 1],
    [1, -3.2, -2.2, -1.4, -1, -1.4, -2.2, -3.2, 1],
    [-4, -3, -2, -1, 0, -1, -2, -3, -4],
    [1, -3.2, -2.2, -1.4, -1, -1.4, -2.2, -3.2, 1],
    [1, -3.6, -2.8, -2.2, -2, -2.2, -2.8, -3.6, 1],
    [1, 1, -3.6, -3.2, -3, -3.2, -3.6, 1, 1],
    [1, 1, 1, 1, -4, 1, 1, 1, 1]
])

# Define necessary paths and constants
ORTHO_FOLDER = Path(r"C:\shrinath_leaf_seg_mrcnn\shrinath\orthomosic\altum\7m")
DEM_FOLDER = Path(r"C:\shrinath_leaf_seg_mrcnn\pipeline 2\new_data\dem")
SLOPE_FOLDER = Path(r'C:\shrinath_leaf_seg_mrcnn\pipeline 2\new_data\a2c')
AREA_FOLDER = Path(r'C:\shrinath_leaf_seg_mrcnn\pipeline 2\new_data\area_coefficent')
shape_plot_path = Path(r'C:\shrinath_leaf_seg_mrcnn\pipeline 2\new_data\gis')
shape_plot_file = "demo_miniplot_scoring_leaf_region_2023.gpkg"
RESULTS_FOLDER_ROOT = Path(r'C:\shrinath_leaf_seg_mrcnn\pipeline 2\new_data\results')
design_miniplot_path = Path(r"C:\shrinath_leaf_seg_mrcnn\pipeline 2\new_data\design_miniplot_path\qgis_labels.xlsx")
INSTANCE_FOLDER = "C:/shrinath_leaf_seg_mrcnn/data2/shrinath"
# pipeline_sources_path = "//192.168.91.193/Dokumente/ifz_daten/COBRI/SOURCES/R/pipeline_sources"
path_model="C:/shrinath_leaf_seg_mrcnn/pipeline 2/new_data/classifiers/"
rdata_file = "C:/shrinath_leaf_seg_mrcnn/pipeline 2/new_data/rscript/Multispectral_indices.csv"
pipeline_path = "C:/shrinath_leaf_seg_mrcnn/pipeline 2/new_data/classifiers/result_dict.json"
MRCNN_ROOT = "C:/shrinath_leaf_seg_mrcnn/Envs/models"
MODEL_DIR = "C:/shrinath_leaf_seg_mrcnn/Envs/models/selected_weights/05_100plus0"
MRCNN_MODEL_NAME = "mask_rcnn_sugarbeat_0050.h5"
MRCNN_MODEL_PATH = os.path.join(MODEL_DIR, MRCNN_MODEL_NAME)
date_id = datetime.now().strftime("%Y%m%d")


# Load the design miniplot
design_miniplot = pd.read_excel(design_miniplot_path, sheet_name="data")

# Load model pipeline configuration
model_label = "wf_022"
with open(pipeline_path, 'r') as f:
    pipeline = json.load(f)
coef_names = pipeline['coef']
class_output = pipeline["class_character"]

# Set constants and parameters
UTMzone = 32
input_image_size = 2400  # Size of images that cut from the big orthomosaic to predict the leaves 
rad = 0.05               # Radius for circle instance
n_ins = 100              # Number of instances
n_ins_test = n_ins * 1.5 # Instances for sampling (LEAF form)
th_prediction = 20       # Minimal percentage of vegetation to sample
min_pix_pred = 30        # Minimal number of pixels of vegetation to sample
th_erosion = 10          # Erosion activation threshold
IMAGING = True           # Option to print images
PRINT_SHAPE_SEL_LEAF = True
FOLDER_DEM_FEATURES = DEM_FOLDER
MAX_AREA_LEAF = 225      # Maximum leaf area in cm2
VERSION_PARAMETERS = "0001"


# Check necessary files
def check_files_exist(paths):
    """Check if all the files in the list exist."""
    for path in paths:
        if not os.path.exists(path):
            raise FileNotFoundError(f"{path} does not exist, PLEASE CHECK FILES/PATHS")
    print("All crucial files checked")

# Define file paths
ortho_file = "20230911.tif"
id_file = ortho_file[:8]
INSTANCE_FORM = "BOTH"
DI_THRESHOLD = 0.50
bands = ("BLUE", "GREEN", "RED", "NIR", "REDEDGE", "LWIR")
file_id_ortho = ortho_file[:8]
ortho_file_path = os.path.join(ORTHO_FOLDER, ortho_file)
dem_file_path = os.path.join(DEM_FOLDER, ortho_file)
shape_plot_full_path = os.path.join(shape_plot_path, shape_plot_file)
# cnn_model_path = "C:\shrinath_leaf_seg_mrcnn\pipeline 2\cnn_model\model_0008-10.177416575833039-0.22589.hdf5" #os.path.join(pipeline['model_path'][0], pipeline['model_list'][0])

# # Check if necessary files exist
check_files_exist([ortho_file_path, dem_file_path])

# Create results folder if it doesn't exist
RESULTS_FOLDER_ROOT = os.path.join(RESULTS_FOLDER_ROOT, date_id)
os.makedirs(RESULTS_FOLDER_ROOT, exist_ok=True)
print(f"Results folder created at: {RESULTS_FOLDER_ROOT}")

# Define additional paths
a2c_path_list = [os.path.join(DEM_FOLDER, "a2c", ortho_file)]
area_coeficient_path_list = [os.path.join(DEM_FOLDER, "area_coeficient", ortho_file)]

# LEAF PROCESSING
INSTANCE_FORM = "LEAF"  # Set to "LEAF" or "BOTH"
shape_leaf_path = os.path.join(INSTANCE_FOLDER, f"1_{file_id_ortho}.gpkg")

# Load the .RData file
Index_List = pd.read_csv(rdata_file)

# Initialize variables
values = None
file = 1

# PROCESSING

# Load sample ortho DEM
print(f"Loading ortho file: {ortho_file_path}")
M1 = rasterio.open(ortho_file_path)

# Load features
slope_path = os.path.join(SLOPE_FOLDER, ortho_file)
if not os.path.exists(slope_path):
    print("Slope file missing. Please calculate slope!")

area_path = os.path.join(AREA_FOLDER, ortho_file)
if not os.path.exists(area_path):
    print("Area coefficient file missing. Please calculate area coefficient!")

with rasterio.open(ortho_file_path) as src:
    profile = src.profile

# Extract data from the ortho and DEM files
rmu_data_t = np.mean(real_res(ortho_file_path, UTMzone))
rmux_data_t = real_res(ortho_file_path, UTMzone)[0]
rmuy_data_t = real_res(ortho_file_path, UTMzone)[1]
D1 = rasterio.open(dem_file_path)
rde_data_t = np.mean(real_res(dem_file_path, UTMzone))

# Load pixel size and slope
pix_size = rasterio.open(area_path)
a2cam = rasterio.open(slope_path)

# Load leaf shape
shape_plot = gpd.read_file(shape_plot_full_path)
shape_plot = shape_plot.to_crs('EPSG:4326')
shape_plot['geometry'] = shape_plot['geometry'].apply(convert_multipolygon_to_polygon)

if 'plot_id' not in shape_plot.columns:
    shape_plot['plot_id'] = range(1, len(shape_plot) + 1)
plot_list_id = sorted(shape_plot['plot_id'])    
# Check and process LEAF instances
LEAF_PROCESSING = False
if INSTANCE_FORM in ["LEAF", "BOTH"]:
    if os.path.exists(shape_leaf_path):
        # Read the shapefile if it exists
        shape_leaf = gpd.read_file(shape_leaf_path)
        print("Shapefile read successfully.")
        LEAF_PROCESSING = True
    else:
        print('Loading the Mask R-CNN Model...')

        # Define configuration for Mask R-CNN
        class CocoSynthConfig(Config):
            NAME = "cocosynth_dataset"
            GPU_COUNT = 1
            IMAGES_PER_GPU = 1
            NUM_CLASSES = 1 + 1  # Background + 1 leaf type
            IMAGE_MIN_DIM = 2048
            IMAGE_MAX_DIM = 2048
            STEPS_PER_EPOCH = 2000
            VALIDATION_STEPS = 100
            BACKBONE = 'resnet101'
            RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)
            TRAIN_ROIS_PER_IMAGE = 32
            MAX_GT_INSTANCES = 50
            POST_NMS_ROIS_TRAINING = 400
            
            

        config = CocoSynthConfig()

        class InferenceConfig(CocoSynthConfig):
            GPU_COUNT = 1
            IMAGES_PER_GPU = 1
            IMAGE_MIN_DIM = 4096
            IMAGE_MAX_DIM = 4096
            DETECTION_MIN_CONFIDENCE = 0.5
            DETECTION_MAX_INSTANCES = 4000
            POST_NMS_ROIS_INFERENCE = 10000

        inference_config = InferenceConfig()
        model = modellib.MaskRCNN(mode="inference", model_dir=MRCNN_ROOT, config=inference_config)
        model.load_weights(MRCNN_MODEL_PATH, by_name=True)
        
        print('Model loaded successfully.')
        leaf_predictions(ORTHO_FOLDER, ortho_file, model, shape_leaf_path,input_image_size)

c:\Users\madde\anaconda3\envs\hhh\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


All crucial files checked
Results folder created at: C:\shrinath_leaf_seg_mrcnn\pipeline 2\new_data\results\20240906
Loading ortho file: C:\shrinath_leaf_seg_mrcnn\shrinath\orthomosic\altum\7m\20230911.tif


c:\Users\madde\anaconda3\envs\hhh\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\shri_code\IFZ\leaf_prediction\utils.py:59: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  UTMxmn, UTMymn = transform(in_proj, out_proj, xmn, ymn)
c:\shri_code\IFZ\leaf_prediction\utils.py:60: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  UTMxmx, UTMymx = transform(in_proj, out_proj, xmx, ymx)
c:\Users\madde\anaconda3\envs\hhh\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' 

Loading the Mask R-CNN Model...
Instructions for updating:
Use fn_output_signature instead
Model loaded successfully.
Creating windows for the predictions
